In [1]:
import torch
import gpytorch


In [2]:
train_x = torch.randn(30000, 1)
kernel = gpytorch.kernels.keops.SpectralGPKernel(train_x, register_latent_params=True)
rhs = torch.randn(30000, 1)

In [3]:
kernel(train_x).matmul(rhs)

SHAPES torch.Size([30000, 1]) torch.Size([30000, 1]) torch.Size([50]) torch.Size([50]) False


tensor([[ -11.5222],
        [-152.2220],
        [ -79.4339],
        ...,
        [-199.6474],
        [  14.2924],
        [-147.1197]])

In [4]:
kernel(train_x).add_jitter(1e-3).inv_matmul(rhs)

SHAPES torch.Size([30000, 1]) torch.Size([30000, 1]) torch.Size([50]) torch.Size([50]) False
Compiling libKeOpstorch022824f86f in /home/wesley/.cache/pykeops-1.3-cpython-37/build-libKeOpstorch022824f86f:
       formula: Sum_Reduction((Sum((((Extract((Cos(((Var(0,1,0) - Var(1,1,2)) * Var(2,50,2))) * Var(3,50,2)),1,49) + Extract((Cos(((Var(0,1,0) - Var(1,1,2)) * Var(2,50,2))) * Var(3,50,2)),0,49)) / Var(4,1,2)) * Var(5,49,2))) * Var(6,1,2)),0)
       aliases: Var(0,1,0); Var(1,1,2); Var(2,50,2); Var(3,50,2); Var(4,1,2); Var(5,49,2); Var(6,1,2); 
       dtype  : float32
... Done.


tensor([[  501.3501],
        [-2036.5688],
        [   -8.4332],
        ...,
        [   50.9966],
        [  213.1179],
        [  -90.3558]])

In [5]:
kernel(train_x).add_jitter(1e-3).inv_matmul(torch.randn(30000, 1))

SHAPES torch.Size([30000, 1]) torch.Size([30000, 1]) torch.Size([50]) torch.Size([50]) False


tensor([[1783.4788],
        [-233.5909],
        [ 796.3500],
        ...,
        [1064.5153],
        [2121.6240],
        [ 961.6417]])